In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.multioutput import MultiOutputRegressor

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error

from scipy.stats import ttest_1samp, uniform, randint, linregress, ttest_ind
from scipy import stats
from statsmodels.stats import weightstats as stests

In [3]:
plt.style.use('seaborn')

In [4]:
%run CoreFunction.ipynb

In [ ]:
TEST_SIZE=0.15
n_comp = 0.95

In [5]:
scaler_min_max = MinMaxScaler()
scaler_z_score = StandardScaler()

In [6]:
def min_max_normalize(data):
    n_data = (data - data.min()) / (data.max() - data.min())
    return n_data

In [7]:
def z_score_normalize(raw_data):
    mean = raw_data.mean(axis=0)
    raw_data -= mean
    std = raw_data.std(axis=0)
    raw_data /= std
    
    return raw_data

In [8]:
load_target, load_attribute, df_data_load = read_data("total load actual",  r"data\df_data_load_ts720_120_24.csv")

In [12]:
data_gmm = df_data_load.iloc[:,1:]

In [13]:
data_gmm_scaled = z_score_normalize(df_data_load.iloc[:,1:])

In [14]:
gmm = GaussianMixture(n_components=2, random_state=0)
gmm.fit(data_gmm_scaled)
labels = gmm.predict(data_gmm_scaled)

In [15]:
data_gmm_scaled_synthetic = gmm.sample(100000)

In [21]:
data_gmm_scaled_synthetic

(array([[ 0.36258676, -0.12467626, -0.00634933, ...,  0.96503246,
          0.60673065,  0.02390058],
        [-0.00897437, -0.87804685, -0.00593373, ...,  0.32604931,
         -0.08723265, -0.37640869],
        [-1.59225596, -0.90283056, -0.00850055, ..., -1.59941331,
         -1.741491  , -1.80668033],
        ...,
        [-0.06904375,  0.0087953 , -0.3031607 , ...,  0.61956147,
          1.00674223,  0.52904412],
        [ 0.41124791, -1.28629258,  4.07915213, ...,  0.26196117,
          0.39777817,  0.30587056],
        [ 0.22475727, -0.09946184, -2.54471637, ...,  0.74180341,
          0.66149189,  0.46011658]]),
 array([0, 0, 0, ..., 1, 1, 1]))

In [24]:
data_gmm_scaled['Cluster'] = labels

In [75]:
df_gmm_scaled_synthetic = pd.DataFrame()
df_gmm_scaled_synthetic

""


In [76]:
df_gmm_scaled_synthetic = pd.DataFrame(data_gmm_scaled_synthetic[0])
df_gmm_scaled_synthetic['Cluster'] = pd.DataFrame(data_gmm_scaled_synthetic[1])
df_gmm_scaled_synthetic

,0,1,2,3,4,5,6,7,8,9,...,910,911,912,913,914,915,916,917,918,Cluster
0,0.362587,-0.124676,-0.006349,-0.358899,-0.897516,-0.054782,-1.187278,0.335053,-0.538798,0.756503,...,-0.058940,-0.221667,0.107838,0.426023,0.952284,1.231598,0.965032,0.606731,0.023901,0
1,-0.008974,-0.878047,-0.005934,-0.933277,-0.129784,0.009539,-1.053568,0.072099,0.616262,1.093898,...,1.445573,1.599844,1.524011,1.449596,1.284516,0.939244,0.326049,-0.087233,-0.376409,0
2,-1.592256,-0.902831,-0.008501,0.961277,-1.044886,-0.165082,-0.710178,0.273046,-0.641658,-1.112846,...,-1.031138,-1.107397,-1.012407,-0.777716,-1.095564,-1.462705,-1.599413,-1.741491,-1.806680,0
3,-2.210053,0.464600,-0.007181,-0.330278,-0.353356,-0.410528,-0.326030,-1.962455,-0.939268,0.064359,...,1.393178,1.604266,1.392944,1.364259,1.356363,1.327297,0.437144,-0.766992,-1.326424,0
4,0.347795,0.378221,-0.005897,-1.629064,-0.417189,-0.128219,0.387213,-1.330657,1.332876,0.081606,...,-0.529383,-1.017714,-1.708101,-2.653540,-2.924269,-2.932268,-2.786715,-2.117952,-1.592560,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.078070,0.587488,-2.181041,0.733940,0.093370,-0.041012,0.688865,-0.037556,-1.767236,0.745817,...,0.395868,0.277564,0.279179,0.493845,0.554831,0.639334,0.667537,0.511289,0.821798,1
99996,-0.736560,0.438029,1.152645,0.984760,-0.100961,-0.603411,-0.296470,0.168024,-1.256651,-0.183501,...,-0.870568,-0.776775,-0.568266,-0.181742,0.253015,0.701094,0.830076,0.620753,0.387962,1
99997,-0.069044,0.008795,-0.303161,0.960342,-0.770059,0.535611,-1.540932,0.441722,-1.365445,-0.047940,...,-0.808636,-1.034675,-0.825558,-0.485041,-0.277082,0.120594,0.619561,1.006742,0.529044,1
99998,0.411248,-1.286293,4.079152,-1.148075,-0.146535,1.160916,-1.730083,1.780057,0.577912,-0.253025,...,-1.340134,-1.367182,-1.296786,-1.104788,-0.662752,-0.179885,0.261961,0.397778,0.305871,1


In [77]:
col_names=data_gmm_scaled.columns.to_list()

In [78]:
df_gmm_scaled_synthetic.columns = [col_names]
df_gmm_scaled_synthetic

,total load actual,weather_main_clouds_Barcelona,weather_main_dust_Barcelona,weather_main_mist_Valencia,weather_main_mist_Bilbao,wind_speed_Bilbao,rain_1h_Barcelona,weather_main_fog_Madrid,pressure_Bilbao,generation other,...,Target_112,Target_113,Target_114,Target_115,Target_116,Target_117,Target_118,Target_119,Target_120,Cluster
0,0.362587,-0.124676,-0.006349,-0.358899,-0.897516,-0.054782,-1.187278,0.335053,-0.538798,0.756503,...,-0.058940,-0.221667,0.107838,0.426023,0.952284,1.231598,0.965032,0.606731,0.023901,0
1,-0.008974,-0.878047,-0.005934,-0.933277,-0.129784,0.009539,-1.053568,0.072099,0.616262,1.093898,...,1.445573,1.599844,1.524011,1.449596,1.284516,0.939244,0.326049,-0.087233,-0.376409,0
2,-1.592256,-0.902831,-0.008501,0.961277,-1.044886,-0.165082,-0.710178,0.273046,-0.641658,-1.112846,...,-1.031138,-1.107397,-1.012407,-0.777716,-1.095564,-1.462705,-1.599413,-1.741491,-1.806680,0
3,-2.210053,0.464600,-0.007181,-0.330278,-0.353356,-0.410528,-0.326030,-1.962455,-0.939268,0.064359,...,1.393178,1.604266,1.392944,1.364259,1.356363,1.327297,0.437144,-0.766992,-1.326424,0
4,0.347795,0.378221,-0.005897,-1.629064,-0.417189,-0.128219,0.387213,-1.330657,1.332876,0.081606,...,-0.529383,-1.017714,-1.708101,-2.653540,-2.924269,-2.932268,-2.786715,-2.117952,-1.592560,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-0.078070,0.587488,-2.181041,0.733940,0.093370,-0.041012,0.688865,-0.037556,-1.767236,0.745817,...,0.395868,0.277564,0.279179,0.493845,0.554831,0.639334,0.667537,0.511289,0.821798,1
99996,-0.736560,0.438029,1.152645,0.984760,-0.100961,-0.603411,-0.296470,0.168024,-1.256651,-0.183501,...,-0.870568,-0.776775,-0.568266,-0.181742,0.253015,0.701094,0.830076,0.620753,0.387962,1
99997,-0.069044,0.008795,-0.303161,0.960342,-0.770059,0.535611,-1.540932,0.441722,-1.365445,-0.047940,...,-0.808636,-1.034675,-0.825558,-0.485041,-0.277082,0.120594,0.619561,1.006742,0.529044,1
99998,0.411248,-1.286293,4.079152,-1.148075,-0.146535,1.160916,-1.730083,1.780057,0.577912,-0.253025,...,-1.340134,-1.367182,-1.296786,-1.104788,-0.662752,-0.179885,0.261961,0.397778,0.305871,1


In [36]:
# df_gmm_scaled_synthetic.to_csv(f'data/df_gmm_scaled_synthetic.csv', index = False, header=True)

In [37]:
# data_gmm_scaled.to_csv(f'data/data_gmm_scaled_origin.csv', index = False, header=True)

In [39]:
# df_data_load_distribution_0 = df_gmm_scaled_synthetic[df_gmm_scaled_synthetic["Cluster"]==0]
# df_data_load_distribution_1 = df_gmm_scaled_synthetic[df_gmm_scaled_synthetic["Cluster"]==1]

In [79]:
df_gmm_scaled_synthetic.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
total load actual,100000.0,0.000080,1.002251,-4.562990,-0.652042,0.045758,0.691903,4.285887
weather_main_clouds_Barcelona,100000.0,0.003013,0.999365,-4.256947,-0.672832,0.000742,0.675489,4.243161
weather_main_dust_Barcelona,100000.0,0.002935,1.003352,-5.634091,-0.009691,-0.007628,-0.005101,6.451650
weather_main_mist_Valencia,100000.0,0.000306,1.004393,-4.771926,-0.630786,-0.010294,0.620664,5.182070
weather_main_mist_Bilbao,100000.0,-0.004175,1.000110,-4.830362,-0.626755,-0.035185,0.588303,5.448766
...,...,...,...,...,...,...,...,...
Target_117,100000.0,-0.001014,0.998913,-4.887915,-0.689011,0.004684,0.688484,4.179936
Target_118,100000.0,-0.000807,0.999655,-5.114429,-0.689094,0.006582,0.695466,3.950265
Target_119,100000.0,-0.000816,1.000629,-4.892402,-0.681078,0.021105,0.703591,4.224362
Target_120,100000.0,-0.002243,1.000392,-4.437383,-0.662629,0.037517,0.697823,4.166947


In [49]:
df_gmm_scaled_synthetic.isnull().sum()

total load actual                0
weather_main_clouds_Barcelona    0
weather_main_dust_Barcelona      0
weather_main_mist_Valencia       0
weather_main_mist_Bilbao         0
                                ..
Target_117                       0
Target_118                       0
Target_119                       0
Target_120                       0
Cluster                          0
Length: 920, dtype: int64

In [80]:
df_gmm_scaled_synthetic.isnull().values.any()

False

In [68]:
df_gmm_scaled_synthetic[df_gmm_scaled_synthetic["Cluster"]==0].isnull().values.any()

True

In [115]:
df_gmm_scaled_synthetic['Cluster'].sum()

Cluster    48720
dtype: int64

##  ШЕГУВАШ ЛИ СЕ!!!!!  :((((((

In [99]:
df_gmm_scaled_synthetic[df_gmm_scaled_synthetic['Cluster']==1]

,total load actual,weather_main_clouds_Barcelona,weather_main_dust_Barcelona,weather_main_mist_Valencia,weather_main_mist_Bilbao,wind_speed_Bilbao,rain_1h_Barcelona,weather_main_fog_Madrid,pressure_Bilbao,generation other,...,Target_112,Target_113,Target_114,Target_115,Target_116,Target_117,Target_118,Target_119,Target_120,Cluster
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
99996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
99997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
99998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
